<a href="https://colab.research.google.com/github/mehmoodosman/financial-analysis-automation-with-LLMs/blob/main/Financial_Analysis_%26_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Financial Analysis & Automation with LLMs Project

# Install Libraries

In [1]:
! pip install yfinance langchain_pinecone openai python-dotenv langchain-community sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.9
    Uninstalling aiohttp-3.11.9:
      Successfully uninstalled aiohttp-3.11.9
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langch

In [2]:
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
import dotenv
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os

In [3]:
def get_stock_info(symbol: str) -> dict:
    """
    Retrieves and formats detailed information about a stock from Yahoo Finance.

    Args:
        symbol (str): The stock ticker symbol to look up.

    Returns:
        dict: A dictionary containing detailed stock information, including ticker, name,
              business summary, city, state, country, industry, and sector.
    """
    data = yf.Ticker(symbol)
    stock_info = data.info

    properties = {
        "Ticker": stock_info.get('symbol', 'Information not available'),
        'Name': stock_info.get('longName', 'Information not available'),
        'Business Summary': stock_info.get('longBusinessSummary', 'Information not available'),
        'City': stock_info.get('city', 'Information not available'),
        'State': stock_info.get('state', 'Information not available'),
        'Country': stock_info.get('country', 'Information not available'),
        'Industry': stock_info.get('industry', 'Information not available'),
        'Sector': stock_info.get('sector', 'Information not available'),
        # Technical Indicators
        "Market Cap": stock_info.get('marketCap', 'Information not available'),
        "Volume": stock_info.get('volume', 'Information not available'),
        "52 Week High": stock_info.get('fiftyTwoWeekHigh', 'Information not available'),
        "52 Week Low": stock_info.get('fiftyTwoWeekLow', 'Information not available'),
        "Dividend Yield": stock_info.get('dividendYield', 'Information not available'),
        "Price to Earnings Ratio (P/E)": stock_info.get('trailingPE', 'Information not available'),
        "Price to Book Ratio (P/B)": stock_info.get('priceToBook', 'Information not available'),
        "Beta": stock_info.get('beta', 'Information not available')
    }

    return properties

In [4]:
data = yf.Ticker("NVDA")
stock_info = data.info
print(stock_info)

{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building and operating metaverse and 3D internet applications. The Compute & Networking segment co

In [5]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for the given text using a specified Hugging Face model.

    Args:
        text (str): The input text to generate embeddings for.
        model_name (str): The name of the Hugging Face model to use.
                          Defaults to "sentence-transformers/all-mpnet-base-v2".

    Returns:
        np.ndarray: The generated embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    """
    Calculates the cosine similarity between two sentences.

    Args:
        sentence1 (str): The first sentence for similarity comparison.
        sentence2 (str): The second sentence for similarity comparison.

    Returns:
        float: The cosine similarity score between the two sentences,
               ranging from -1 (completely opposite) to 1 (identical).

    Notes:
        Prints the similarity score to the console in a formatted string.
    """
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    similarity_score = similarity[0][0]
    print(f"Cosine similarity between the two sentences: {similarity_score:.4f}")
    return similarity_score


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like running to the office"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Cosine similarity between the two sentences: 0.6133


In [6]:
aapl_info = get_stock_info('AAPL')
print(aapl_info)

{'Ticker': 'AAPL', 'Name': 'Apple Inc.', 'Business Summary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experien

In [7]:
aapl_description = aapl_info['Business Summary']

company_description = "I want to find companies that make smartphones and are headquarted in California"

similarity = cosine_similarity_between_sentences(aapl_description, company_description)

Cosine similarity between the two sentences: 0.3635


# Get all the Stocks in the Stock Market

First, we need to get the symbols (also known as tickers) of all the stocks in the stock market


In [8]:
def get_company_tickers():
    """
    Downloads and parses the Stock ticker symbols from the GitHub-hosted SEC company tickers JSON file.

    Returns:
        dict: A dictionary containing company tickers and related information.

    Notes:
        The data is sourced from the official SEC website via a GitHub repository:
        https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json
    """
    # URL to fetch the raw JSON file from GitHub
    url = "https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json"

    # Making a GET request to the URL
    response = requests.get(url)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parse the JSON content directly
        company_tickers = json.loads(response.content.decode('utf-8'))

        # Optionally save the content to a local file for future use
        with open("company_tickers.json", "w", encoding="utf-8") as file:
            json.dump(company_tickers, file, indent=4)

        print("File downloaded successfully and saved as 'company_tickers.json'")
        return company_tickers
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

company_tickers = get_company_tickers()

File downloaded successfully and saved as 'company_tickers.json'


In [9]:
company_tickers

{'0': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'},
 '1': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'},
 '2': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'},
 '3': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'},
 '4': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'},
 '5': {'cik_str': 1326801, 'ticker': 'META', 'title': 'Meta Platforms, Inc.'},
 '6': {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'},
 '7': {'cik_str': 1067983,
  'ticker': 'BRK-B',
  'title': 'BERKSHIRE HATHAWAY INC'},
 '8': {'cik_str': 1046179,
  'ticker': 'TSM',
  'title': 'TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD'},
 '9': {'cik_str': 1730168, 'ticker': 'AVGO', 'title': 'Broadcom Inc.'},
 '10': {'cik_str': 59478, 'ticker': 'LLY', 'title': 'ELI LILLY & Co'},
 '11': {'cik_str': 19617, 'ticker': 'JPM', 'title': 'JPMORGAN CHASE & CO'},
 '12': {'cik_str': 104169, 'ticker': 'WMT', 'title': 'Walmart Inc.'},
 '13': {'cik_str'

In [10]:
len(company_tickers)

9998

# Inserting Stocks into Pinecone

**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "stocks" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-12-02 at 4 48 03 PM](https://github.com/user-attachments/assets/13b484da-cd00-4337-a4db-4779080eb42c)


**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**


![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)




In [11]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "stocks"
namespace = "stock-descriptions"

hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

<ipython-input-11-8d5c0f356cf6>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings()
<ipython-input-11-8d5c0f356cf6>:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()


# Sequential Processing

It will take very long to process all the stocks like this!

[![](https://mermaid.ink/img/pako:eNqNkl1rgzAUhv9KyMXYwF74cSVjYA0rhY6WKRQWe5FqWqU1cUm8GKX_ffmwa65Gc6Hn5LxvzmM8F1jzhsIUHgUZWlCiigG9MlwoXp_AqpPqdS_esmyzCsBivV7o10fxXgagLFbZDsxmb2COi44dzxRsBK-plDoBZSsoaXbuNPeU4941qWBBv0fKVEfOvud5yejh0NWdLr1U0LnMmts2eYhDMLsZgEHa3TV5aEXbEG-zZZnasiXfLMGnaScVeNKRHDiT1DNunRGF5pMFtUaAiCKe5h4hp84jHHks9mJ8mMjBRBOMrT9G45wommis84bjYThZHuPYwzA_xqeIHUU8UZjyYxDOiOIJwhj_uRKnzhOc3FHsdHgoiUNJJhRTfgzFGVEyoRijj0JZAwPYU9GTrtFjfDHbFVQt7WkFUx02RJzMMF21joyKFz-shqkSIw2g4OOxhemBnKXOxqEhiqKO6DHt_3YHwr44v-XXX7It6B4?type=png)](https://mermaid.live/edit#pako:eNqNkl1rgzAUhv9KyMXYwF74cSVjYA0rhY6WKRQWe5FqWqU1cUm8GKX_ffmwa65Gc6Hn5LxvzmM8F1jzhsIUHgUZWlCiigG9MlwoXp_AqpPqdS_esmyzCsBivV7o10fxXgagLFbZDsxmb2COi44dzxRsBK-plDoBZSsoaXbuNPeU4941qWBBv0fKVEfOvud5yejh0NWdLr1U0LnMmts2eYhDMLsZgEHa3TV5aEXbEG-zZZnasiXfLMGnaScVeNKRHDiT1DNunRGF5pMFtUaAiCKe5h4hp84jHHks9mJ8mMjBRBOMrT9G45wommis84bjYThZHuPYwzA_xqeIHUU8UZjyYxDOiOIJwhj_uRKnzhOc3FHsdHgoiUNJJhRTfgzFGVEyoRijj0JZAwPYU9GTrtFjfDHbFVQt7WkFUx02RJzMMF21joyKFz-shqkSIw2g4OOxhemBnKXOxqEhiqKO6DHt_3YHwr44v-XXX7It6B4)

In [ ]:
for idx, stock in company_tickers.items():
    stock_ticker = stock['ticker']
    stock_data = get_stock_info(stock_ticker)
    stock_description = stock_data['Business Summary']

    print(f"Processing stock {idx} / {len(company_tickers)} :", stock_ticker)

    vectorstore_from_documents = PineconeVectorStore.from_documents(
        documents=[Document(page_content=stock_description, metadata=stock_data)],
        embedding=hf_embeddings,
        index_name=index_name,
        namespace=namespace
    )

Processing stock 0 / 9998 : NVDA
Processing stock 1 / 9998 : AAPL
Processing stock 2 / 9998 : MSFT


KeyboardInterrupt: 

# Parallelizing

[![](https://mermaid.ink/img/pako:eNqFk0uLgzAQgP_KkMOe7MHXRZaC1baXvsDCwqqHrGarVJMSE9hS-983NnWxULceRmf4Pp2MyQVlLCfIQweOTwXsw4SCuvw4Eiw7wqpsxPsXn_r-bmXAcrtdqts6WuwN2Ecr3wB__blJYTKZwixe45JCwKjgrKoI7zwXdphjlVXwwfiR8CbVH9BxdjPbqKxlJTAlTDbVuYXAjDUNZveSHWcZaRromkj_F61etIbire8Xpt2b9tDslvpCdHrRGYrddF6Ibi-6D4vsBjqcUWDe_NCMl0TAG6gfwwmEWOA7FlgasEYBWwP2KOBowBkFXA24Y4COoW61DRklLcwvQUHUHooEFrK53hHrAbkX7WdF51nRfVLUcX4fs8y6ObawMON-phvyI2CGRVakyEA14TUuc7XnL52ZIFGQmiTIU4855scEJfSqOCwFi840Q57gkhiIM3kokPeNq0Zl8pRjQcISq4NT_1VPmH4y1ufXXwdkCM8?type=png)](https://mermaid.live/edit#pako:eNqFk0uLgzAQgP_KkMOe7MHXRZaC1baXvsDCwqqHrGarVJMSE9hS-983NnWxULceRmf4Pp2MyQVlLCfIQweOTwXsw4SCuvw4Eiw7wqpsxPsXn_r-bmXAcrtdqts6WuwN2Ecr3wB__blJYTKZwixe45JCwKjgrKoI7zwXdphjlVXwwfiR8CbVH9BxdjPbqKxlJTAlTDbVuYXAjDUNZveSHWcZaRromkj_F61etIbire8Xpt2b9tDslvpCdHrRGYrddF6Ibi-6D4vsBjqcUWDe_NCMl0TAG6gfwwmEWOA7FlgasEYBWwP2KOBowBkFXA24Y4COoW61DRklLcwvQUHUHooEFrK53hHrAbkX7WdF51nRfVLUcX4fs8y6ObawMON-phvyI2CGRVakyEA14TUuc7XnL52ZIFGQmiTIU4855scEJfSqOCwFi840Q57gkhiIM3kokPeNq0Zl8pRjQcISq4NT_1VPmH4y1ufXXwdkCM8)

In [12]:
# Initialize tracking lists
successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")

Loaded 7795 successful tickers
Loaded 919 unsuccessful tickers


In [13]:
def process_stock(stock_ticker: str) -> str:
    # Skip if already processed
    if stock_ticker in successful_tickers:
        return f"Already processed {stock_ticker}"

    try:
        # Get and store stock data
        stock_data = get_stock_info(stock_ticker)
        stock_description = stock_data['Business Summary']

        # Store stock description in Pinecone
        vectorstore_from_texts = PineconeVectorStore.from_documents(
            documents=[Document(page_content=stock_description, metadata=stock_data)],
            embedding=hf_embeddings,
            index_name=index_name,
            namespace=namespace,
            ids=[stock_ticker],
        )

        # Track success
        with open('successful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        successful_tickers.append(stock_ticker)

        return f"Processed {stock_ticker} successfully"

    except Exception as e:
        # Track failure
        with open('unsuccessful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        unsuccessful_tickers.append(stock_ticker)

        return f"ERROR processing {stock_ticker}: {e}"

def parallel_process_stocks(tickers: list, max_workers: int = 10) -> None:
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {
            executor.submit(process_stock, ticker): ticker
            for ticker in tickers
        }

        for future in concurrent.futures.as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                result = future.result()
                print(result)

                # Stop on error
                if result.startswith("ERROR"):
                    print(f"Stopping program due to error in {ticker}")
                    executor.shutdown(wait=False)
                    raise SystemExit(1)

            except Exception as exc:
                print(f'{ticker} generated an exception: {exc}')
                print("Stopping program due to exception")
                executor.shutdown(wait=False)
                raise SystemExit(1)

# Prepare your tickers
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

# Process them
parallel_process_stocks(tickers_to_process, max_workers=10)

Streaming output truncated to the last 5000 lines.
Already processed ABIT
Already processed PFGC
Already processed KFY
Already processed DVA
Already processed LION
Already processed CHRW
Already processed TPH
Already processed CYBR
Already processed BELFA
Already processed AKAM
Already processed RELY
Already processed TXRH
Already processed LNC
Already processed CNA
Already processed SHC
Already processed SUZ
Already processed SUPN
Already processed SWK
Already processed IMVT
Already processed UHS
Already processed SOR
Already processed VEL
Already processed TPR
Already processed IGT
Already processed UNM
Already processed CLH
Already processed CNK
Already processed OMVKY
Already processed NABL
Already processed RVTY
Already processed ASGN
Already processed RNR
Already processed TRNO
Already processed GLPI
Already processed REG
Already processed SWKS
Already processed GMS
Already processed QCRH
Already processed TSQ
Already processed SBC
Already processed SYRE
Already processed DNOW
Al

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DAIC%20?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DAIC+&crumb=doQHsxdBsJB


Processed DAIC  successfully
Processed CPHC successfully
Processed IIPR-PA successfully
Processed IRET successfully
Processed GAM-PB successfully
Processed GTN-A successfully
Processed PTCHF successfully
Processed FMCKP successfully
Processed DRDGF successfully
Processed BELFB successfully
Processed AGM-PD successfully
Processed CMRE-PB successfully
Processed CMRE-PC successfully
Processed MEOBF successfully
Processed CMRE-PD successfully
Processed KBSR successfully
Processed KELYB successfully
Processed BH successfully
Processed HLTC successfully
Processed SMBMF successfully
Processed GGT-PE successfully
Processed CELJF successfully
Processed DEFTF successfully
Processed HVT-A successfully
Processed MPSYF successfully
Processed GAMI successfully
Processed UMH-PD successfully
Processed ECCX successfully
Processed ATROB successfully
Processed DGICB successfully
Processed GGN-PB successfully
Processed ARTNB successfully
Processed MITT-PA successfully
Processed MITT-PB successfully
Proces

ERROR:yfinance:500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/THCPW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=THCPW&crumb=doQHsxdBsJB


Processed CEROW successfully
Processed KEY-PL successfully
Processed LFT-PA successfully
Processed IRAAU successfully
Processed UNOV successfully
Processed ADNWW successfully
Processed IRAAW successfully
Processed THCPU successfully
Processed THCPW successfully
Processed VLYPN successfully
Processed MHNC successfully
Processed ARQQW successfully
Processed MSCF successfully
Processed SCCC successfully
Processed SCCD successfully
Processed DSAQW successfully
Processed SCCE successfully
Processed SCCG successfully
Processed SACC successfully
Processed SCCF successfully
Processed GDL-PC successfully
Processed SACH-PA successfully
Processed BLEUR successfully
Processed OUST-WT successfully
Processed BLEUU successfully
Processed BLEUW successfully
Processed DSAQU successfully
Processed CTLPP successfully
Processed GFAIW successfully
Processed TEN-PF successfully
Processed OUST-WTA successfully
Processed TEN-PE successfully
Processed NMPWP successfully
Processed NMKCP successfully
Processed N

...

Processed FVNNR successfully
Processed FVNNU successfully
Processed FITBO successfully
Processed NVAWW successfully
Processed FITBP successfully
Processed NVAAF successfully
Processed CHEB-WT successfully
Processed DMYY-WT successfully
Processed TOIIW successfully
Processed CHEB-UN successfully
Processed DMYY-UN successfully
Processed ATEK-WT successfully
Processed LUNRW successfully
Processed IONQ-WT successfully
Processed ATEK-UN successfully
Processed XFOWW successfully
Processed TBMCR successfully
Processed AGXRW successfully
Processed NMHIW successfully
Processed BURUW successfully
Processed VAL-WT successfully
Processed MDNC successfully
Processed HSPOR successfully
Processed GDEVW successfully
Processed CRTDW successfully
Processed HSPOW successfully
Processed GLTK successfully
Processed HSPOU successfully
Processed AP-WT successfully
Processed ABLLL successfully
Processed HUDAR successfully
Processed ABLLW successfully
Processed HUDAU successfully
Processed RCFA-WT successfully

ERROR:yfinance:500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LUCYW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LUCYW&crumb=doQHsxdBsJB


Processed BYNOU successfully
Processed BYNOW successfully
Processed NEMCL successfully
Processed UHGI successfully
Processed DYCQU successfully
Processed DYCQR successfully
Processed LUCYW successfully
Processed ASB-PF successfully
Processed BRIPF successfully
Processed BIPJ successfully
Processed VFSWW successfully
Processed MNESP successfully
Processed BIPI successfully
Processed BIPH successfully
Processed MNQFF successfully
Processed ATHS successfully
Processed STRRP successfully
Processed JOCM successfully
Processed CLDT-PA successfully
Processed HYZNW successfully
Processed MNLCF successfully
Processed MNUFF successfully
Processed BKKT-WT successfully
Processed BIP-PA successfully
Processed BIP-PB successfully
Processed SAIHW successfully
Processed ATH-PB successfully
Processed ATH-PC successfully
Processed SIMAW successfully
Processed SIMAU successfully
Processed ATH-PD successfully
Processed ATH-PE successfully
Processed ATLCL successfully
Processed ATLCZ successfully
Processed

ERROR:yfinance:500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PFTAW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PFTAW&crumb=doQHsxdBsJB


Processed WHLRL successfully
Processed ESGLW successfully
Processed INPAP successfully
Processed PFTAW successfully
Processed PFTAU successfully
Processed COF-PJ successfully
Processed COF-PI successfully
Processed LIFWW successfully
Processed LIFWZ successfully
Processed COF-PL successfully
Processed COF-PK successfully
Processed COF-PN successfully
Processed PSPX successfully
Processed EPDU successfully
Processed CRESW successfully
Processed SBEV-WT successfully
Processed LMMY successfully
Processed DHAIW successfully
Processed MYPSW successfully
Processed AMBI-WT successfully
Processed PROCW successfully
Processed BFRIW successfully
Processed BMTX-WT successfully
Processed BWVTF successfully
Processed ATMP successfully
Processed TFC-PI successfully
Processed TFC-PR successfully
Processed DJP successfully
Processed TFC-PO successfully
Processed VXZ successfully
Processed COWTF successfully
Processed EVVAQ successfully
Processed VXX successfully
Processed DTSTW successfully
Processed 

# Perform RAG

In [14]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [15]:
query = "What are some companies that manufacture consumer hardware?"

In [16]:
raw_query_embedding = get_huggingface_embeddings(query)

In [17]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

In [18]:
top_matches

{'matches': [{'id': 'KE',
              'metadata': {'52 Week High': 27.73,
                           '52 Week Low': 16.64,
                           'Beta': 1.261,
                           'Business Summary': 'Kimball Electronics, Inc. '
                                               'engages in the provision of '
                                               'electronics manufacturing, '
                                               'engineering, and supply chain '
                                               'support services to customers '
                                               'in the automotive, medical, '
                                               'and industrial end markets. '
                                               'The Company also offers '
                                               'contract manufacturing '
                                               'services, including '
                                               'engineering and suppl

In [19]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [20]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [21]:
print(augmented_query)

<CONTEXT>
Kimball Electronics, Inc. engages in the provision of electronics manufacturing, engineering, and supply chain support services to customers in the automotive, medical, and industrial end markets. The Company also offers contract manufacturing services, including engineering and supply chain support for the production of electronic assemblies and other products, including non-electronic components, medical devices, medical disposables, and precision molded plastics, as well as automation, test, and inspection equipment primarily used in automotive, medical, and industrial applications. In addition, it is also involved in the production and testing of printed circuit board assemblies, as well as clean room assembly, cold chain, and product sterilization management activities. Further, the company offers design services and support, supply chain services and support, rapid prototyping and new product introduction support, product design and process validation and qualification,

# Setting up Groq for RAG

1. Get your Groq API Key [here](https://console.groq.com/keys). Through Groq, you get free access to various LLMs

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)

In [23]:
!pip install groq
from groq import Groq

client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.6 MB/s eta 0:00:00


In [24]:
system_prompt = f"""You are an expert at providing answers about stocks. Please answer my question provided.
"""

chat_completion = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)
response = chat_completion.choices[0].message.content

In [25]:
print(response)

Based on the provided context, some companies that manufacture consumer hardware include:

1. Deswell Industries, Inc. - They produce plastic components for consumer electronic entertainment products, such as telephones, printers, and scanners.

2. Forward Industries, Inc. - They design and manufacture carrying and protective solutions for consumer electronic products, including cases and accessories for medical monitoring kits, sporting and recreational products, and other portable electronic and non-electronic products.

3. Best Buy Co., Inc. - They sell a wide range of consumer electronics and home appliances, but note that they also have some in-house brands that manufacture consumer hardware, such as Geek Squad and Best Buy Brands, but it is not explicitly stated in the context.

Additionally, other companies mentioned in the context may also produce consumer hardware components or products, but it's not explicitly stated. These include companies such as:
- 3M Company, in their Tr